In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import time

import march_madness as mm
import random

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures

In [2]:
analysis = mm.Analysis()

In [3]:
analysis.load_seasons()

In [4]:
analysis.seasons.keys()

dict_keys([1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022])

In [5]:
analysis.calc_seasons_features()

1985


/Users/jacobhow/Documents/march-madness-ML/march_madness.py:765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{t}TR'] = df[f'{t}OR'] + df[f'{t}DR']
/Users/jacobhow/Documents/march-madness-ML/march_madness.py:766: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{t}FGM2'] = df[f'{t}FGM'] - df[f'{t}FGM3']
/Users/jacobhow/Documents/march-madness-ML/march_madness.py:767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

1986


/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/jacobhow/Documents/march-madness-ML/march_madness.py:765: SettingWithCopyWarning: 
A value is trying to be set on

1987


...

/Users/jacobhow/Documents/march-madness-ML/march_madness.py:765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{t}TR'] = df[f'{t}OR'] + df[f'{t}DR']
/Users/jacobhow/Documents/march-madness-ML/march_madness.py:766: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{t}FGM2'] = df[f'{t}FGM'] - df[f'{t}FGM3']
/Users/jacobhow/Documents/march-madness-ML/march_madness.py:767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

2021


/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return

2022


/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return

In [94]:
seasons = np.arange(1985,2022)

# Phase 2 - Detailed Game Results

Phase 2 of the model training is using the 'detailed game results' for each game that gives additional box score stats beyond the final score of each game.

These stats included the follwing for both teams:
* FGM - field goals made
* FGA - field goals attempted
* FGM3 - three pointers made
* FGA3 - three pointers attempted
* FTM - free throws made
* FTA - free throws attempted
* OR - offensive rebounds
* DR - defensive rebounds
* Ast - assists
* TO - turnovers committed
* Stl - steals
* Blk - blocks
* PF - personal fouls committed

## Detailed Game Results

These statistics were then used to generate additional statistics such as offensive and defensive efficiency, various rates (ex. steals per possesion), and relavent ratios (ex. ratio of 3pt attempts to all field attempts). The game averages of these statistics for each team and their opponents were calculated and added to the model.

In [95]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','avg win margin','std win margin','avg loss margin',
                'std loss margin','capped avg win margin','capped std win margin','capped avg loss margin',
                'capped std loss margin','close wins','close losses','weighted top64 wins','weighted top32 wins',
                'weighted top16 wins','weighted top8 wins','weighted top64 losses','weighted top32 losses',
                'weighted top16 losses','weighted top8 losses','last10 win pct','last10 weighted win pct',
                'last5 win pct','last5 weighted win pct','conference tourney wins','conference champ','Team Avg FGM',
                'Team Avg FGA','Team Avg FGM3','Team Avg FGA3','Team Avg FTM','Team Avg FTA',
                'Team Avg OR','Team Avg DR','Team Avg Ast','Team Avg TO%','Team Avg Stl%','Team Avg Blk%',
                'Team Avg PF','Team Avg TR','Team Avg FGM2','Team Avg FGA2','Team Avg FG%','Team Avg FG2%',
                'Team Avg FG3%','Team Avg FGA3%','Team Avg FT%','Team Avg Pos','Team Avg OEff','Opp Avg FGM',
                'Opp Avg FGA','Opp Avg FGM3','Opp Avg FGA3','Opp Avg FTM','Opp Avg FTA','Opp Avg OR','Opp Avg DR',
                'Opp Avg Ast','Opp Avg TO%','Opp Avg Stl%','Opp Avg Blk%','Opp Avg PF','Opp Avg TR','Opp Avg FGM2',
                'Opp Avg FGA2','Opp Avg FG%','Opp Avg FG2%','Opp Avg FG3%','Opp Avg FGA3%','Opp Avg FT%',
                'Opp Avg Pos','Opp Avg OEff']

In [96]:
X, y = analysis.seasons_generate_tourney_model_data(seasons=seasons,feature_keys=feature_keys)

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=100)

In [98]:
X_train.shape

(1737, 148)

In [99]:
y_train.shape

(1737,)

In [100]:
X_test.shape

(580, 148)

In [101]:
y_test.shape

(580,)

In [102]:
logreg = LogisticRegression().fit(X_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [103]:
logreg.score(X_train,y_train)

0.7357512953367875

In [104]:
logreg.score(X_test,y_test)

0.7086206896551724

In [105]:
forest = RandomForestClassifier(n_estimators=1000,max_features='sqrt',max_depth=5,random_state=100).fit(X_train,y_train)

In [106]:
forest.score(X_train,y_train)

0.7996545768566494

In [107]:
forest.score(X_test,y_test)

0.7

In [108]:
logreg_pred = mm.bound_predictions(logreg.predict_proba(X_test)[:,1])

In [109]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [110]:
analysis.score_model_predictions(y_test,logreg_pred)

0.5500592529775931

In [111]:
analysis.score_model_predictions(y_test,forest_pred)

0.5613021703413059

## Detailed Game Results + Interactions

Additionally, the interactions between all of these features were added to the model. This appeared to help the Random Forest but actually hurt the Logistic Regression.

In [112]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','avg win margin','std win margin','avg loss margin',
                'std loss margin','capped avg win margin','capped std win margin','capped avg loss margin',
                'capped std loss margin','close wins','close losses','weighted top64 wins','weighted top32 wins',
                'weighted top16 wins','weighted top8 wins','weighted top64 losses','weighted top32 losses',
                'weighted top16 losses','weighted top8 losses','last10 win pct','last10 weighted win pct',
                'last5 win pct','last5 weighted win pct','conference tourney wins','conference champ','Team Avg FGM','Team Avg FGA','Team Avg FGM3','Team Avg FGA3','Team Avg FTM','Team Avg FTA',
                'Team Avg OR','Team Avg DR','Team Avg Ast','Team Avg TO%','Team Avg Stl%','Team Avg Blk%',
                'Team Avg PF','Team Avg TR','Team Avg FGM2','Team Avg FGA2','Team Avg FG%','Team Avg FG2%',
                'Team Avg FG3%','Team Avg FGA3%','Team Avg FT%','Team Avg Pos','Team Avg OEff','Opp Avg FGM',
                'Opp Avg FGA','Opp Avg FGM3','Opp Avg FGA3','Opp Avg FTM','Opp Avg FTA','Opp Avg OR','Opp Avg DR',
                'Opp Avg Ast','Opp Avg TO%','Opp Avg Stl%','Opp Avg Blk%','Opp Avg PF','Opp Avg TR','Opp Avg FGM2',
                'Opp Avg FGA2','Opp Avg FG%','Opp Avg FG2%','Opp Avg FG3%','Opp Avg FGA3%','Opp Avg FT%',
                'Opp Avg Pos','Opp Avg OEff']

In [113]:
X, y = analysis.seasons_generate_tourney_model_data(seasons=seasons,feature_keys=feature_keys)

In [114]:
X.shape

(2317, 148)

In [115]:
poly = PolynomialFeatures(2)

In [116]:
X = poly.fit_transform(X)

In [117]:
X.shape

(2317, 11175)

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=100)

In [119]:
X_train.shape

(1737, 11175)

In [120]:
y_train.shape

(1737,)

In [121]:
X_test.shape

(580, 11175)

In [122]:
y_test.shape

(580,)

In [123]:
logreg = LogisticRegression().fit(X_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [124]:
logreg.score(X_train,y_train)

0.7472654001151411

In [125]:
logreg.score(X_test,y_test)

0.6775862068965517

In [126]:
forest = RandomForestClassifier(n_estimators=1000,max_features='sqrt',max_depth=5,random_state=100).fit(X_train,y_train)

In [127]:
forest.score(X_train,y_train)

0.8388025331030512

In [128]:
forest.score(X_test,y_test)

0.7120689655172414

In [129]:
logreg_pred = mm.bound_predictions(logreg.predict_proba(X_test)[:,1])

In [130]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [131]:
analysis.score_model_predictions(y_test,logreg_pred)

0.5726628934270334

In [132]:
analysis.score_model_predictions(y_test,forest_pred)

0.5537268178615626

## Net Detailed Game Results

The last features I engineer were the 'net' statistics for all these parameters. These parameters can be interpreted as "how the team did compared to what the opponent generally allowed". For example if an opponent allows an average of 5 three-pointers per game and the team scored 10, that would be a net of 5 for that parameter in that game. 

So on offense these metrics capture, how well did the team do compared to what their opponents allow on average.

And on defense they capture, how well did you limit your opponent compared to how much they normally produce.

'Net' parameters were calculated for all stats derived from the 'detailed' information and were the average difference for all of the team's games.

In [133]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','avg win margin','std win margin','avg loss margin',
                'std loss margin','capped avg win margin','capped std win margin','capped avg loss margin',
                'capped std loss margin','close wins','close losses','weighted top64 wins','weighted top32 wins',
                'weighted top16 wins','weighted top8 wins','weighted top64 losses','weighted top32 losses',
                'weighted top16 losses','weighted top8 losses','last10 win pct','last10 weighted win pct',
                'last5 win pct','last5 weighted win pct','conference tourney wins','conference champ','Team Avg FGM','Team Avg FGA','Team Avg FGM3','Team Avg FGA3','Team Avg FTM','Team Avg FTA',
                'Team Avg OR','Team Avg DR','Team Avg Ast','Team Avg TO%','Team Avg Stl%','Team Avg Blk%',
                'Team Avg PF','Team Avg TR','Team Avg FGM2','Team Avg FGA2','Team Avg FG%','Team Avg FG2%',
                'Team Avg FG3%','Team Avg FGA3%','Team Avg FT%','Team Avg Pos','Team Avg OEff','Opp Avg FGM',
                'Opp Avg FGA','Opp Avg FGM3','Opp Avg FGA3','Opp Avg FTM','Opp Avg FTA','Opp Avg OR','Opp Avg DR',
                'Opp Avg Ast','Opp Avg TO%','Opp Avg Stl%','Opp Avg Blk%','Opp Avg PF','Opp Avg TR','Opp Avg FGM2',
                'Opp Avg FGA2','Opp Avg FG%','Opp Avg FG2%','Opp Avg FG3%','Opp Avg FGA3%','Opp Avg FT%',
                'Opp Avg Pos','Opp Avg OEff','Net Team Avg FGM','Net Team Avg FGA','Net Team Avg FGM3',
                'Net Team Avg FGA3','Net Team Avg FTM','Net Team Avg FTA','Net Team Avg OR','Net Team Avg DR',
                'Net Team Avg Ast','Net Team Avg TO%','Net Team Avg Stl%','Net Team Avg Blk%','Net Team Avg PF',
                'Net Team Avg TR','Net Team Avg FGM2','Net Team Avg FGA2','Net Team Avg FG%','Net Team Avg FG2%',
                'Net Team Avg FG3%','Net Team Avg FGA3%','Net Team Avg FT%','Net Team Avg Pos','Net Team Avg OEff',
                'Net Opp Avg FGM','Net Opp Avg FGA','Net Opp Avg FGM3','Net Opp Avg FGA3','Net Opp Avg FTM',
                'Net Opp Avg FTA','Net Opp Avg OR','Net Opp Avg DR','Net Opp Avg Ast','Net Opp Avg TO%',
                'Net Opp Avg Stl%','Net Opp Avg Blk%','Net Opp Avg PF','Net Opp Avg TR','Net Opp Avg FGM2',
                'Net Opp Avg FGA2','Net Opp Avg FG%','Net Opp Avg FG2%','Net Opp Avg FG3%','Net Opp Avg FGA3%',
                'Net Opp Avg FT%','Net Opp Avg Pos','Net Opp Avg OEff']

In [134]:
X, y = analysis.seasons_generate_tourney_model_data(seasons=seasons,feature_keys=feature_keys)

In [135]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=100)

In [136]:
X_train.shape

(1737, 240)

In [137]:
y_train.shape

(1737,)

In [138]:
X_test.shape

(580, 240)

In [139]:
y_test.shape

(580,)

In [140]:
logreg = LogisticRegression().fit(X_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [141]:
logreg.score(X_train,y_train)

0.7380541162924582

In [142]:
logreg.score(X_test,y_test)

0.6948275862068966

In [143]:
forest = RandomForestClassifier(n_estimators=1000,max_features='sqrt',max_depth=5,random_state=100).fit(X_train,y_train)

In [144]:
forest.score(X_train,y_train)

0.8025331030512378

In [145]:
forest.score(X_test,y_test)

0.7051724137931035

In [146]:
logreg_pred = mm.bound_predictions(logreg.predict_proba(X_test)[:,1])

In [147]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [148]:
analysis.score_model_predictions(y_test,logreg_pred)

0.5554635928737769

In [149]:
analysis.score_model_predictions(y_test,forest_pred)

0.5663653609026418

## Net Detailed Game Results + Interactions

Again, the interactions between all of these features were added to the model. This again, helped the Random Forest and hurt the Logistic Regression.

In [150]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','avg win margin','std win margin','avg loss margin',
                'std loss margin','capped avg win margin','capped std win margin','capped avg loss margin',
                'capped std loss margin','close wins','close losses','weighted top64 wins','weighted top32 wins',
                'weighted top16 wins','weighted top8 wins','weighted top64 losses','weighted top32 losses',
                'weighted top16 losses','weighted top8 losses','last10 win pct','last10 weighted win pct',
                'last5 win pct','last5 weighted win pct','conference tourney wins','conference champ','Team Avg FGM','Team Avg FGA','Team Avg FGM3','Team Avg FGA3','Team Avg FTM','Team Avg FTA',
                'Team Avg OR','Team Avg DR','Team Avg Ast','Team Avg TO%','Team Avg Stl%','Team Avg Blk%',
                'Team Avg PF','Team Avg TR','Team Avg FGM2','Team Avg FGA2','Team Avg FG%','Team Avg FG2%',
                'Team Avg FG3%','Team Avg FGA3%','Team Avg FT%','Team Avg Pos','Team Avg OEff','Opp Avg FGM',
                'Opp Avg FGA','Opp Avg FGM3','Opp Avg FGA3','Opp Avg FTM','Opp Avg FTA','Opp Avg OR','Opp Avg DR',
                'Opp Avg Ast','Opp Avg TO%','Opp Avg Stl%','Opp Avg Blk%','Opp Avg PF','Opp Avg TR','Opp Avg FGM2',
                'Opp Avg FGA2','Opp Avg FG%','Opp Avg FG2%','Opp Avg FG3%','Opp Avg FGA3%','Opp Avg FT%',
                'Opp Avg Pos','Opp Avg OEff','Net Team Avg FGM','Net Team Avg FGA','Net Team Avg FGM3',
                'Net Team Avg FGA3','Net Team Avg FTM','Net Team Avg FTA','Net Team Avg OR','Net Team Avg DR',
                'Net Team Avg Ast','Net Team Avg TO%','Net Team Avg Stl%','Net Team Avg Blk%','Net Team Avg PF',
                'Net Team Avg TR','Net Team Avg FGM2','Net Team Avg FGA2','Net Team Avg FG%','Net Team Avg FG2%',
                'Net Team Avg FG3%','Net Team Avg FGA3%','Net Team Avg FT%','Net Team Avg Pos','Net Team Avg OEff',
                'Net Opp Avg FGM','Net Opp Avg FGA','Net Opp Avg FGM3','Net Opp Avg FGA3','Net Opp Avg FTM',
                'Net Opp Avg FTA','Net Opp Avg OR','Net Opp Avg DR','Net Opp Avg Ast','Net Opp Avg TO%',
                'Net Opp Avg Stl%','Net Opp Avg Blk%','Net Opp Avg PF','Net Opp Avg TR','Net Opp Avg FGM2',
                'Net Opp Avg FGA2','Net Opp Avg FG%','Net Opp Avg FG2%','Net Opp Avg FG3%','Net Opp Avg FGA3%',
                'Net Opp Avg FT%','Net Opp Avg Pos','Net Opp Avg OEff']

In [151]:
X, y = analysis.seasons_generate_tourney_model_data(seasons=seasons,feature_keys=feature_keys)

In [152]:
X.shape

(2317, 240)

In [153]:
poly = PolynomialFeatures(2)

In [154]:
X = poly.fit_transform(X)

In [155]:
X.shape

(2317, 29161)

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=100)

In [157]:
X_train.shape

(1737, 29161)

In [158]:
y_train.shape

(1737,)

In [159]:
X_test.shape

(580, 29161)

In [160]:
y_test.shape

(580,)

In [161]:
logreg = LogisticRegression().fit(X_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [162]:
logreg.score(X_train,y_train)

0.7754749568221071

In [163]:
logreg.score(X_test,y_test)

0.6775862068965517

In [164]:
forest = RandomForestClassifier(n_estimators=1000,max_features='sqrt',max_depth=5,random_state=100).fit(X_train,y_train)

In [165]:
forest.score(X_train,y_train)

0.8434081750143926

In [166]:
forest.score(X_test,y_test)

0.7120689655172414

In [167]:
logreg_pred = mm.bound_predictions(logreg.predict_proba(X_test)[:,1])

In [168]:
forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])

In [169]:
analysis.score_model_predictions(y_test,logreg_pred)

0.6228954561393399

In [170]:
analysis.score_model_predictions(y_test,forest_pred)

0.5563214100075714

# Model Tuning Tests

Finally I tested the effect of tuning the key model parameters on the performance of the Random Forest. The two parameters tested were the number of trees in the forest and the 'mad_depth' allowed for the trees.

### Feature Set - Net Detailed Game Results + Interactions

In [171]:
feature_keys = ['tourney seed','weighted win pct','owp','oowp','avg win margin','std win margin','avg loss margin',
                'std loss margin','capped avg win margin','capped std win margin','capped avg loss margin',
                'capped std loss margin','close wins','close losses','weighted top64 wins','weighted top32 wins',
                'weighted top16 wins','weighted top8 wins','weighted top64 losses','weighted top32 losses',
                'weighted top16 losses','weighted top8 losses','last10 win pct','last10 weighted win pct',
                'last5 win pct','last5 weighted win pct','conference tourney wins','conference champ','Team Avg FGM','Team Avg FGA','Team Avg FGM3','Team Avg FGA3','Team Avg FTM','Team Avg FTA',
                'Team Avg OR','Team Avg DR','Team Avg Ast','Team Avg TO%','Team Avg Stl%','Team Avg Blk%',
                'Team Avg PF','Team Avg TR','Team Avg FGM2','Team Avg FGA2','Team Avg FG%','Team Avg FG2%',
                'Team Avg FG3%','Team Avg FGA3%','Team Avg FT%','Team Avg Pos','Team Avg OEff','Opp Avg FGM',
                'Opp Avg FGA','Opp Avg FGM3','Opp Avg FGA3','Opp Avg FTM','Opp Avg FTA','Opp Avg OR','Opp Avg DR',
                'Opp Avg Ast','Opp Avg TO%','Opp Avg Stl%','Opp Avg Blk%','Opp Avg PF','Opp Avg TR','Opp Avg FGM2',
                'Opp Avg FGA2','Opp Avg FG%','Opp Avg FG2%','Opp Avg FG3%','Opp Avg FGA3%','Opp Avg FT%',
                'Opp Avg Pos','Opp Avg OEff','Net Team Avg FGM','Net Team Avg FGA','Net Team Avg FGM3',
                'Net Team Avg FGA3','Net Team Avg FTM','Net Team Avg FTA','Net Team Avg OR','Net Team Avg DR',
                'Net Team Avg Ast','Net Team Avg TO%','Net Team Avg Stl%','Net Team Avg Blk%','Net Team Avg PF',
                'Net Team Avg TR','Net Team Avg FGM2','Net Team Avg FGA2','Net Team Avg FG%','Net Team Avg FG2%',
                'Net Team Avg FG3%','Net Team Avg FGA3%','Net Team Avg FT%','Net Team Avg Pos','Net Team Avg OEff',
                'Net Opp Avg FGM','Net Opp Avg FGA','Net Opp Avg FGM3','Net Opp Avg FGA3','Net Opp Avg FTM',
                'Net Opp Avg FTA','Net Opp Avg OR','Net Opp Avg DR','Net Opp Avg Ast','Net Opp Avg TO%',
                'Net Opp Avg Stl%','Net Opp Avg Blk%','Net Opp Avg PF','Net Opp Avg TR','Net Opp Avg FGM2',
                'Net Opp Avg FGA2','Net Opp Avg FG%','Net Opp Avg FG2%','Net Opp Avg FG3%','Net Opp Avg FGA3%',
                'Net Opp Avg FT%','Net Opp Avg Pos','Net Opp Avg OEff']

In [172]:
X, y = analysis.seasons_generate_tourney_model_data(seasons=seasons,feature_keys=feature_keys)

In [173]:
poly = PolynomialFeatures(2)

In [174]:
X = poly.fit_transform(X)

In [175]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=100)

In [176]:
X_train.shape

(1737, 29161)

In [177]:
y_train.shape

(1737,)

In [178]:
X_test.shape

(580, 29161)

In [179]:
y_test.shape

(580,)

## Number of Trees

Test model performance as a function of the number of trees

In [180]:
number_trees = [100,200,400,800,1600,3200,6400,12800]

In [181]:
for n_estimators in number_trees:
    start_time = time.time()
    forest = RandomForestClassifier(n_estimators=n_estimators,max_features='sqrt',max_depth=5,random_state=100).fit(X_train,y_train)
    end_time = time.time()
    training_score = forest.score(X_train,y_train)
    test_score = forest.score(X_test,y_test)
    forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])
    competition_score = analysis.score_model_predictions(y_test,forest_pred)
    training_time = end_time - start_time
    
    print(f'Number of Trees: {n_estimators}')
    print(f'  Training Set Score : {training_score:.3f}')
    print(f'     Test Set Score  : {test_score:.3f}')
    print(f'   Competition Score : {competition_score:.3f}')
    print(f'      Training Time  : {training_time:.2f}')

Number of Trees: 100
  Training Set Score : 0.838
     Test Set Score  : 0.709
   Competition Score : 0.556
      Training Time  : 5.60
Number of Trees: 200
  Training Set Score : 0.838
     Test Set Score  : 0.712
   Competition Score : 0.555
      Training Time  : 11.01
Number of Trees: 400
  Training Set Score : 0.843
     Test Set Score  : 0.707
   Competition Score : 0.555
      Training Time  : 21.87
Number of Trees: 800
  Training Set Score : 0.843
     Test Set Score  : 0.709
   Competition Score : 0.556
      Training Time  : 43.38
Number of Trees: 1600
  Training Set Score : 0.845
     Test Set Score  : 0.714
   Competition Score : 0.556
      Training Time  : 86.58
Number of Trees: 3200
  Training Set Score : 0.843
     Test Set Score  : 0.710
   Competition Score : 0.556
      Training Time  : 173.57
Number of Trees: 6400
  Training Set Score : 0.845
     Test Set Score  : 0.716
   Competition Score : 0.556
      Training Time  : 345.99
Number of Trees: 12800
  Training Set

## Max Depth

Test Model Performance for fixed number of trees and variable 'max_depth'

In [182]:
max_depths = [3,5,7,9,11]

In [183]:
for max_depth in max_depths:
    forest = RandomForestClassifier(n_estimators=1600,
                                    max_features='sqrt',max_depth=max_depth,random_state=100).fit(X_train,y_train)
    training_score = forest.score(X_train,y_train)
    test_score = forest.score(X_test,y_test)
    forest_pred = mm.bound_predictions(forest.predict_proba(X_test)[:,1])
    competition_score = analysis.score_model_predictions(y_test,forest_pred)
    
    print(f'Max Depth: {max_depth}')
    print(f'  Training Set Score : {training_score:.3f}')
    print(f'     Test Set Score  : {test_score:.3f}')
    print(f'   Competition Score : {competition_score:.3f}')

Max Depth: 3
  Training Set Score : 0.744
     Test Set Score  : 0.714
   Competition Score : 0.559
Max Depth: 5
  Training Set Score : 0.845
     Test Set Score  : 0.714
   Competition Score : 0.556
Max Depth: 7
  Training Set Score : 0.927
     Test Set Score  : 0.702
   Competition Score : 0.559
Max Depth: 9
  Training Set Score : 0.963
     Test Set Score  : 0.688
   Competition Score : 0.562
Max Depth: 11
  Training Set Score : 0.984
     Test Set Score  : 0.688
   Competition Score : 0.562
